In [1]:
import flask

In [1]:
379885467838-c19jsii830g3pbsdhts5m9egsp3s7h95.apps.googleusercontent.com

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

     |████████████████████████████████| 203 kB 1.4 MB/s eta 0:00:01


In [4]:
! pip install flask_login

# OAuth code

In [ ]:
from flask import Flask
import pymongo, json
from bson import ObjectId
from bson import json_util
import database_config as cfg
from user import User

from flask import Flask, render_template, url_for, redirect, request
from authlib.integrations.flask_client import OAuth
from flask_login import (
    LoginManager,
    current_user,
    login_required,
    login_user,
    logout_user,
)

'''
Required App, Oauth, Session Management Initializations
'''

app = Flask(__name__)

oauth = OAuth(app)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.needs_refresh_message = (u"Session timedout, please re-login")
login_manager.needs_refresh_message_category = "info"


'''
Read credentials from the configuration file
'''
app.config['SECRET_KEY'] = cfg.google_client_credentials['secret_key']
app.config['GOOGLE_CLIENT_ID'] = cfg.google_client_credentials['client_id']
app.config['GOOGLE_CLIENT_SECRET'] = cfg.google_client_credentials['client_secret']


'''
Initialize google oauth registration details
'''
google = oauth.register(
    name = 'google',
    client_id = app.config["GOOGLE_CLIENT_ID"],
    client_secret = app.config["GOOGLE_CLIENT_SECRET"],
    access_token_url = 'https://accounts.google.com/o/oauth2/token',
    access_token_params = None,
    authorize_url = 'https://accounts.google.com/o/oauth2/auth',
    authorize_params = None,
    api_base_url = 'https://www.googleapis.com/oauth2/v1/',
    userinfo_endpoint = 'https://openidconnect.googleapis.com/v1/userinfo',  # This is only needed if using openId to fetch user info
    client_kwargs = {'scope': 'openid email profile'},
)



@login_manager.user_loader
def loadUser(user_id):
    '''
    Load current user on the basis of the Unique ID
    '''
    return User.get(user_id)


@app.route("/logout")
@login_required
def logout():
    '''
    If the user clicks on the logout link
    '''
    logout_user()
    return redirect(url_for("index"))


@app.before_request
def beforeRequest():
    '''
    Setting logout time
    '''
    session.permanent = True
    app.permanent_session_lifetime = timedelta(minutes=5)
    
    
@app.route("/")
def index():
    '''
    Link to the homepage. This is what the UI will be calling.
    '''
    
    if current_user.is_authenticated:
        
        return (
            "<p>Hello, {}! You're logged in! Email: {}</p>"
            '<a class="button" href="/logout">Logout</a>'.format(
                current_user.name, current_user.email
            )
        )
    else:
        return '<a class="button" href="/login">Google Login</a>'
    
###Get parameters and make things work
@app.route("/get_cab_price"):
def run_main():
    source = request.args.get('source')
    destination = request.args.get('destination')
    type_of_car = request.args.get('cab_type')
    
    # Call google maps API - Get code from Shubha
    
    
    # Call uber and lyft API
    
    
    # Lyft not available
    
    # Call weather API
    
    
    
@app.route('/login')
def googleLogin():
    '''
    Google login route
    '''
    
    google = oauth.create_client('google')
    redirected_url = url_for('google_authorize', _external=True)
    return google.authorize_redirect(redirected_url)


@app.route('/login/google/authorize')
def googleAuthorize():
    '''
    Google authorization route
    '''
    
    google = oauth.create_client('google')
    token = google.authorize_access_token()
    user_info = google.get('userinfo').json()
    print (user_info['name'], user_info['email'])
    
    if user_info.get("verified_email"):
        unique_id = user_info["id"]
        user_email = user_info["email"]
        user_name = user_info["name"]
    else:
        return "User email not available or not verified by Google.", 400
    
    
    '''
    Check for user's existence in the database, insert one if not already in the database
    '''
    user = User.get(unique_id)
    
    if user is not None:
        login_user(user)
    else:
#         user=User(unique_id, user_name, user_email, ['nanavati', 'ruby'])
#         user_created = user.create(unique_id, user_name, user_email, ['nanavati', 'ruby'])
#         if user_created == False:
#             return "User creation in the database failed"

    print ("Google Login Successful")
    # Send user to the required page
    return redirect(url_for("index"))


def createDatabaseConnection():
    '''
    Create MongoClient Connection and return the database
    '''
    
    mongo_client = pymongo.MongoClient(cfg.mongo_credentials['hostIP'])
    argus_database = mongo_client[cfg.mongo_credentials['databaseName']]
    
    return argus_database


@app.route("/getDatasetDetails/<dataset_name>")
def fetchDatasetDetails(dataset_name):
    '''
    Create Database Connection and query the collection with dsapp responses, return as a list and the total image count
    '''
    
    argus_database = createDatabaseConnection()
    
    collection_of_dsapp_responses = argus_database[cfg.mongo_credentials['datasetDetailsCollection']]
    list_of_responses = collection_of_dsapp_responses.find({'dataset_name':dataset_name})
    
    list_of_responses = list(list_of_responses)
    total_image_count = len(list_of_responses)
    
    return {"Total Images": str(total_image_count), "Responses_list":json.loads(json_util.dumps(list_of_responses))}



if __name__ == '__main__':
    app.run()


In [2]:
import requests
import json
api_key = "58004db0930cf9524c8308063af9b2ff"
lat = "48.208176"
lon = "16.373819"
url = "https://api.openweathermap.org/data/2.5/onecall?lat=%s&lon=%s&appid=%s&units=metric" % (lat, lon, api_key)
response = requests.get(url)
data = json.loads(response.text)
print(data)

{'lat': 48.2082, 'lon': 16.3738, 'timezone': 'Europe/Vienna', 'timezone_offset': 3600, 'current': {'dt': 1646178933, 'sunrise': 1646199269, 'sunset': 1646239154, 'temp': -1.24, 'feels_like': -5.89, 'pressure': 1028, 'humidity': 60, 'dew_point': -7.25, 'uvi': 0, 'clouds': 0, 'visibility': 10000, 'wind_speed': 4.12, 'wind_deg': 330, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}]}, 'minutely': [{'dt': 1646178960, 'precipitation': 0}, {'dt': 1646179020, 'precipitation': 0}, {'dt': 1646179080, 'precipitation': 0}, {'dt': 1646179140, 'precipitation': 0}, {'dt': 1646179200, 'precipitation': 0}, {'dt': 1646179260, 'precipitation': 0}, {'dt': 1646179320, 'precipitation': 0}, {'dt': 1646179380, 'precipitation': 0}, {'dt': 1646179440, 'precipitation': 0}, {'dt': 1646179500, 'precipitation': 0}, {'dt': 1646179560, 'precipitation': 0}, {'dt': 1646179620, 'precipitation': 0}, {'dt': 1646179680, 'precipitation': 0}, {'dt': 1646179740, 'precipitation': 0}, {'dt': 

In [3]:
data['current']

{'dt': 1646178933,
 'sunrise': 1646199269,
 'sunset': 1646239154,
 'temp': -1.24,
 'feels_like': -5.89,
 'pressure': 1028,
 'humidity': 60,
 'dew_point': -7.25,
 'uvi': 0,
 'clouds': 0,
 'visibility': 10000,
 'wind_speed': 4.12,
 'wind_deg': 330,
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01n'}]}

In [15]:
! pip install uber-rides

     |████████████████████████████████| 57 kB 1.0 MB/s eta 0:00:01


In [14]:
class Jessie:
    def __init__(self):
        self.hobby='Statistics'
    
    def change_hobby(self, hobby='Probability'):
        self.hobby=hobby
    

In [15]:
j=Jessie()

In [16]:
j.hobby

'Statistics'

In [19]:
j.change_hobby('Viz')

In [20]:
j.hobby

'Viz'

In [53]:
from flask_login import UserMixin
import pandas as pd


class User(UserMixin):
    '''
    The user class of type Usermixin from flask_login handles the user information.
    param: Oauth unique ID, name, email
    methods: get(user_id): Checks if the user exists in the database
             create(user_id, name, email): Creates a new user in the database
    '''
    
    def __init__(self, id_, name, email):
        self.id = id_
        self.name = name
        self.email = email

    @staticmethod
    def get(user_id):
        '''
        This module checks if the user exists in the database.
        param: user_id
        return : Object of the User class with required details
        '''
        
        user_information=pd.read_csv('data/users.csv')
        specific_user_details=user_information.loc[user_information['user_id']==user_id]

        return User(specific_user_details['user_id'][0], specific_user_details['name'][0], specific_user_details['email'][0])

    @staticmethod
    def create(id_, name, email):
        '''
        Creates a user if not already present in the database. 
        param: google oauth user_id, name, email
        return: True if successfully added to the database
        return: False if any error while adding to the database
        '''        
        try:
            user_information=pd.read_csv('data/users.csv')
            current_user = pd.DataFrame({'user_id':[id_], 'name':[name], 'email':[email]})

            user_information = pd.concat([user_information, current_user])
            user_information.to_csv('data/users.csv', index=None)
            return True
        except: 
            print ("User creation failed")
            return False
        


In [54]:
User.get('116296017614486242589')

116296017614486242589


In [38]:
df=pd.read_csv('data/users.csv')

In [39]:
df.head()

,user_id,name,email


In [42]:
id_='116296017614486242589'
name='Rohit Lokwani'
email='rlokwani@uw.edu'